In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Pará - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Pará - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade,Pará - Consumo de Cimento (t)
0,2008,0.653838,8.343245e+07,4.475828e+06,9.446969,7.069042e+07,8.116333,0.631139,5.483335e+08,2.546644e+09,...,1172.693755,35.946490,1008.509202,4848.884846,27.010379,23.052703,80.170200,1.147919,0.762850,1559.491000
1,2009,0.653797,8.418073e+07,4.507563e+06,9.465568,7.086672e+07,8.148790,0.634850,5.302764e+08,2.679945e+09,...,1310.052663,36.120974,1008.356065,3110.216649,26.957054,23.034986,80.332906,1.075145,0.764286,1355.544000
2,2010,0.653639,8.675706e+07,4.555784e+06,9.542119,7.180858e+07,8.112071,0.638688,5.046425e+08,2.891930e+09,...,1443.581473,36.824311,1008.325688,2155.562456,26.838317,22.916120,80.307326,1.001921,0.765262,1640.682000
3,2011,0.653365,8.890935e+07,4.574157e+06,9.572272,7.235179e+07,8.099477,0.642653,4.854622e+08,3.091988e+09,...,1536.079376,37.794614,1008.488595,1742.787440,26.780148,22.842507,80.217883,0.944877,0.765786,1856.176000
4,2012,0.652914,9.072117e+07,4.559887e+06,9.574043,7.265948e+07,9.368278,0.646103,4.867661e+08,3.189506e+09,...,1569.763418,37.800824,1008.648003,1488.532092,26.730543,22.851410,80.485786,0.886023,0.765851,2042.910000
5,2013,0.652490,9.239028e+07,4.525097e+06,9.578467,7.295531e+07,9.857304,0.651362,5.225848e+08,3.093872e+09,...,1509.357990,36.963360,1008.858495,1334.355904,26.733107,22.837087,80.447713,0.829758,0.765422,2318.608000
6,2014,0.651864,9.370048e+07,4.468711e+06,9.571191,7.308848e+07,10.244256,0.656308,5.676413e+08,2.779340e+09,...,1427.236076,36.496952,1009.067756,1232.551825,26.867590,22.874833,80.065651,0.788621,0.764707,2041.730714
7,2015,0.651186,9.493439e+07,4.396932e+06,9.582867,7.332896e+07,10.537304,0.659084,5.179392e+08,2.561731e+09,...,1386.312966,36.539513,1009.084655,1168.300890,26.935981,22.950086,80.146850,0.763383,0.763682,1863.549771
8,2016,0.652132,9.599328e+07,4.320024e+06,9.607059,7.362401e+07,10.587975,0.667623,4.524144e+08,2.516859e+09,...,1292.085356,38.175099,1009.021941,1146.435512,26.885491,22.912033,80.211878,0.737811,0.771424,1705.680514
9,2017,0.652690,9.681084e+07,4.313294e+06,9.643875,7.397420e+07,10.306831,0.670044,4.213689e+08,2.497710e+09,...,1128.424890,41.723496,1009.076536,1076.508642,26.811809,22.889590,80.405874,0.723978,0.770280,1564.940000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pará - IDH,Pará - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
0,0.653838,8.343245e+07,4.475828e+06,9.446969,7.069042e+07,8.116333,0.631139,5.483335e+08,2.546644e+09,1.874883e+10,...,0.680030,1172.693755,35.946490,1008.509202,4848.884846,27.010379,23.052703,80.170200,1.147919,0.762850
1,0.653797,8.418073e+07,4.507563e+06,9.465568,7.086672e+07,8.148790,0.634850,5.302764e+08,2.679945e+09,2.055015e+10,...,0.681704,1310.052663,36.120974,1008.356065,3110.216649,26.957054,23.034986,80.332906,1.075145,0.764286
2,0.653639,8.675706e+07,4.555784e+06,9.542119,7.180858e+07,8.112071,0.638688,5.046425e+08,2.891930e+09,2.342247e+10,...,0.683252,1443.581473,36.824311,1008.325688,2155.562456,26.838317,22.916120,80.307326,1.001921,0.765262
3,0.653365,8.890935e+07,4.574157e+06,9.572272,7.235179e+07,8.099477,0.642653,4.854622e+08,3.091988e+09,2.604265e+10,...,0.684681,1536.079376,37.794614,1008.488595,1742.787440,26.780148,22.842507,80.217883,0.944877,0.765786
4,0.652914,9.072117e+07,4.559887e+06,9.574043,7.265948e+07,9.368278,0.646103,4.867661e+08,3.189506e+09,2.708753e+10,...,0.685697,1569.763418,37.800824,1008.648003,1488.532092,26.730543,22.851410,80.485786,0.886023,0.765851
5,0.652490,9.239028e+07,4.525097e+06,9.578467,7.295531e+07,9.857304,0.651362,5.225848e+08,3.093872e+09,2.523394e+10,...,0.687339,1509.357990,36.963360,1008.858495,1334.355904,26.733107,22.837087,80.447713,0.829758,0.765422
6,0.651864,9.370048e+07,4.468711e+06,9.571191,7.308848e+07,10.244256,0.656308,5.676413e+08,2.779340e+09,1.954361e+10,...,0.688700,1427.236076,36.496952,1009.067756,1232.551825,26.867590,22.874833,80.065651,0.788621,0.764707
7,0.651186,9.493439e+07,4.396932e+06,9.582867,7.332896e+07,10.537304,0.659084,5.179392e+08,2.561731e+09,1.327596e+10,...,0.689092,1386.312966,36.539513,1009.084655,1168.300890,26.935981,22.950086,80.146850,0.763383,0.763682
8,0.652132,9.599328e+07,4.320024e+06,9.607059,7.362401e+07,10.587975,0.667623,4.524144e+08,2.516859e+09,1.011370e+10,...,0.694712,1292.085356,38.175099,1009.021941,1146.435512,26.885491,22.912033,80.211878,0.737811,0.771424
9,0.652690,9.681084e+07,4.313294e+06,9.643875,7.397420e+07,10.306831,0.670044,4.213689e+08,2.497710e+09,8.456309e+09,...,0.695413,1128.424890,41.723496,1009.076536,1076.508642,26.811809,22.889590,80.405874,0.723978,0.770280


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1355.544000
1     1640.682000
2     1856.176000
3     2042.910000
4     2318.608000
5     2041.730714
6     1863.549771
7     1705.680514
8     1564.940000
9     1375.281000
10    1290.928000
11    1444.547000
12    1529.052000
13    1592.624000
14            NaN
Name: Pará - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pará - IDH,Pará - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
0,1.324222,-1.759256,0.299586,-1.981848,-1.870624,-1.353160,-1.576092,1.204752,-0.990234,0.166091,...,-1.566920,-0.626188,-0.863255,-0.967463,2.803264,1.936402,1.658039,-0.761793,2.002801,-1.354813
1,1.274511,-1.602011,0.622544,-1.695638,-1.713888,-1.320968,-1.312905,0.820657,-0.546688,0.428911,...,-1.253172,0.028137,-0.808861,-1.487639,1.218869,1.318238,1.421067,-0.213337,1.493766,-0.784667
2,1.081041,-1.060624,1.113275,-0.517673,-0.876518,-1.357388,-1.040702,0.275392,0.158672,0.847995,...,-0.962925,0.664217,-0.589599,-1.590821,0.348922,-0.058194,-0.168808,-0.299564,0.981589,-0.397217
3,0.745928,-0.608343,1.300248,-0.053680,-0.393573,-1.369878,-0.759421,-0.132596,0.824346,1.230291,...,-0.695012,1.104842,-0.287112,-1.037461,-0.027227,-0.732514,-1.153417,-0.601063,0.582576,-0.188840
4,0.193042,-0.227609,1.155025,-0.026427,-0.120026,-0.111436,-0.514766,-0.104860,1.148829,1.382743,...,-0.504495,1.265299,-0.285176,-0.495989,-0.258923,-1.307543,-1.034340,0.301999,0.170908,-0.163203
5,-0.326024,0.123137,0.800981,0.041651,0.142988,0.373598,-0.141809,0.657046,0.830615,1.112297,...,-0.196768,0.977551,-0.546251,0.219007,-0.399419,-1.277823,-1.225919,0.173660,-0.222645,-0.333625
6,-1.092457,0.398461,0.227158,-0.070314,0.261380,0.757390,0.209035,1.615453,-0.215960,0.282052,...,0.058490,0.586354,-0.691651,0.929824,-0.492190,0.281147,-0.721038,-1.114213,-0.510386,-0.617540
7,-1.922759,0.657755,-0.503318,0.109359,0.475181,1.048046,0.405897,0.558230,-0.940033,-0.632425,...,0.131924,0.391412,-0.678383,0.987226,-0.550740,1.073956,0.285498,-0.840505,-0.686921,-1.024405
8,-0.764233,0.880270,-1.285989,0.481628,0.737493,1.098303,1.011528,-0.835563,-1.089341,-1.093812,...,1.185430,-0.057452,-0.168497,0.774201,-0.570665,0.488663,-0.223484,-0.621304,-0.865789,2.050181
9,-0.081515,1.052072,-1.354473,1.048147,1.048832,0.819454,1.183185,-1.495939,-1.153056,-1.335633,...,1.316861,-0.837068,0.937698,0.959647,-0.634387,-0.365485,-0.523664,0.032626,-0.962547,1.596109


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[-0.92654349,  1.00199625, -1.06025747,  1.30207234,
          1.10478641,  0.65941712,  1.15634775, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          1.07439526, -1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     1355.544000
1     1640.682000
2     1856.176000
3     2042.910000
4     2318.608000
5     2041.730714
6     1863.549771
7     1705.680514
8     1564.940000
9     1375.281000
10    1290.928000
11    1444.547000
Name: Pará - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    1444.547
Name: Pará - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pará - IDH,Pará - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
12,-0.819242,0.814133,-0.996062,1.239044,0.97042,0.595754,1.119834,1.108405,2.635823,-0.189959,...,0.911811,-1.695891,1.553086,0.035714,-0.656689,-0.219776,2.805206,3.026913,-1.023926,-0.677741


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-0.81924176,  0.81413298, -0.99606183,  1.23904422,
          0.97041981,  0.59575364,  1.11983447,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          0.91181081, -1

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1529.052
Name: Pará - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [21]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[891125274, 292666291, 1867266667, 1434112926, 3581014660, 1893333659, 2965907667, 4030598625, 1271483820, 2249518503, 2807887625, 3683373509, 307472343, 3000935402, 1227981205, 1961017919, 153707596, 2087233711, 2927727532, 2161560292, 660787049, 1303442843, 3549132917, 1860823040, 1972902414, 484546995, 3582952136, 1807801629, 1826393989, 4164621341, 2941833029, 3523165868, 330419901, 3790487047, 3310950855, 1440849158, 1658181746, 590786484, 2302042955, 4241964059, 3039180029, 3441444418, 3981296288, 3718048205, 2947748228, 606735704, 470809019, 591350041, 3430685188, 345943463, 390287887, 268031083, 2641912582, 2316729631, 2127123853, 3316042448, 117718006, 1234113320, 921405874, 2944516561, 3087377626, 2275444356, 2204018406, 789233000, 2919610982, 129333815, 2681281899, 2578189939, 205698082, 1141046047, 2491101914, 729780795, 2526676686, 88536283, 470189924, 4141405668, 330481075, 1813413049, 181242921, 226771989, 2430470359, 1442236026, 657714986, 646926611, 1382858180, 1570767

loss: 16421.751953125


Step: 91 ___________________________________________
loss: 24183.896484375


Step: 92 ___________________________________________
loss: 18027.806640625


Step: 93 ___________________________________________
loss: 20584.876953125


Step: 94 ___________________________________________
loss: 7519.953125


Step: 95 ___________________________________________
loss: 17050.828125


Step: 96 ___________________________________________
loss: 15758.8037109375


Step: 97 ___________________________________________
loss: 22119.787109375


Step: 98 ___________________________________________
loss: 25321.248046875


Step: 99 ___________________________________________
loss: 15593.7236328125


final_seed: 390287887


In [22]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 3014018.0000
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 2884190.7500
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 1477638.5000
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 522369.7500
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 1513039.0000
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 218647.0781
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 236618.2031
Epoch 8/10000
1/1 [==============================] - 0s 3ms/step - loss: 150361.8281
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 93643.6641
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 90065.4375
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 42074.9023
Epoch 12/10000
1/1 [==============================] - 0s 4ms/st

1/1 [==============================] - 0s 4ms/step - loss: 46793.0508
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 56286.5195
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 72261.0391
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 69128.2422
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 62232.8711
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 75573.1953
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 45134.2148
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 40246.3789
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 73639.1641
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 131147.4844
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 30785.6875
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - lo

1/1 [==============================] - 0s 4ms/step - loss: 46424.6602
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 49915.4570
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 74033.9375
Epoch 195/10000
1/1 [==============================] - 0s 6ms/step - loss: 62184.3945
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 56024.0625
Epoch 197/10000
1/1 [==============================] - 0s 5ms/step - loss: 80132.0703
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 62932.3867
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 71967.6328
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 64064.9844
Epoch 201/10000
1/1 [==============================] - 0s 5ms/step - loss: 60972.3086
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - loss: 47091.6367
Epoch 203/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 84493.9844
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 51000.9414
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 65141.6719
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 43876.2852
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 47747.5156
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 31665.8965
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 63635.3320
Epoch 294/10000
1/1 [==============================] - 0s 5ms/step - loss: 62701.4961
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 93437.0625
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 70823.9609
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 61270.0312
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 58536.9570
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 56905.0742
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 101872.7266
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 73783.2812
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 55614.5898
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 54009.4805
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 63142.4258
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 56509.4805
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 43462.2773
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 51889.5117
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 54175.3633
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - 

...

1/1 [==============================] - 0s 4ms/step - loss: 17060.6387
Epoch 2175/10000
1/1 [==============================] - 0s 4ms/step - loss: 21862.0801
Epoch 2176/10000
1/1 [==============================] - 0s 4ms/step - loss: 26936.7324
Epoch 2177/10000
1/1 [==============================] - 0s 4ms/step - loss: 23509.5801
Epoch 2178/10000
1/1 [==============================] - 0s 4ms/step - loss: 31665.7480
Epoch 2179/10000
1/1 [==============================] - 0s 4ms/step - loss: 19864.2812
Epoch 2180/10000
1/1 [==============================] - 0s 4ms/step - loss: 40285.8438
Epoch 2181/10000
1/1 [==============================] - 0s 4ms/step - loss: 24641.0938
Epoch 2182/10000
1/1 [==============================] - 0s 4ms/step - loss: 32419.6172
Epoch 2183/10000
1/1 [==============================] - 0s 4ms/step - loss: 20576.2637
Epoch 2184/10000
1/1 [==============================] - 0s 4ms/step - loss: 32637.1797
Epoch 2185/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 13403.8477
Epoch 2269/10000
1/1 [==============================] - 0s 4ms/step - loss: 42683.6914
Epoch 2270/10000
1/1 [==============================] - 0s 4ms/step - loss: 18534.6035
Epoch 2271/10000
1/1 [==============================] - 0s 4ms/step - loss: 39512.7500
Epoch 2272/10000
1/1 [==============================] - 0s 4ms/step - loss: 36115.7539
Epoch 2273/10000
1/1 [==============================] - 0s 4ms/step - loss: 18491.3496
Epoch 2274/10000
1/1 [==============================] - 0s 4ms/step - loss: 20324.7090
Epoch 2275/10000
1/1 [==============================] - 0s 4ms/step - loss: 17441.0527
Epoch 2276/10000
1/1 [==============================] - 0s 4ms/step - loss: 27523.7285
Epoch 2277/10000
1/1 [==============================] - 0s 4ms/step - loss: 29318.4531
Epoch 2278/10000
1/1 [==============================] - 0s 4ms/step - loss: 39819.3281
Epoch 2279/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 24594.7051
Epoch 2363/10000
1/1 [==============================] - 0s 5ms/step - loss: 7284.1753
Epoch 2364/10000
1/1 [==============================] - 0s 5ms/step - loss: 45224.7617
Epoch 2365/10000
1/1 [==============================] - 0s 4ms/step - loss: 34409.8633
Epoch 2366/10000
1/1 [==============================] - 0s 4ms/step - loss: 34743.4805
Epoch 2367/10000
1/1 [==============================] - 0s 4ms/step - loss: 50668.7617
Epoch 2368/10000
1/1 [==============================] - 0s 4ms/step - loss: 36989.6445
Epoch 2369/10000
1/1 [==============================] - 0s 4ms/step - loss: 40234.4570
Epoch 2370/10000
1/1 [==============================] - 0s 4ms/step - loss: 38080.3945
Epoch 2371/10000
1/1 [==============================] - 0s 4ms/step - loss: 43907.2773
Epoch 2372/10000
1/1 [==============================] - 0s 4ms/step - loss: 23129.4004
Epoch 2373/10000
1/1 [==============================] - 0s 4m

1/1 [==============================] - 0s 4ms/step - loss: 39387.8320
Epoch 2457/10000
1/1 [==============================] - 0s 5ms/step - loss: 20265.9336
Epoch 2458/10000
1/1 [==============================] - 0s 4ms/step - loss: 27540.6973
Epoch 2459/10000
1/1 [==============================] - 0s 4ms/step - loss: 43255.6836
Epoch 2460/10000
1/1 [==============================] - 0s 4ms/step - loss: 23524.4531
Epoch 2461/10000
1/1 [==============================] - 0s 4ms/step - loss: 42888.7930
Epoch 2462/10000
1/1 [==============================] - 0s 4ms/step - loss: 21529.8691
Epoch 2463/10000
1/1 [==============================] - 0s 5ms/step - loss: 41415.0586
Epoch 2464/10000
1/1 [==============================] - 0s 5ms/step - loss: 20104.1992
Epoch 2465/10000
1/1 [==============================] - 0s 4ms/step - loss: 34462.1328
Epoch 2466/10000
1/1 [==============================] - 0s 4ms/step - loss: 12872.0049
Epoch 2467/10000
1/1 [==============================] - 0s 5

1/1 [==============================] - 0s 4ms/step - loss: 26070.4629
Epoch 2551/10000
1/1 [==============================] - 0s 4ms/step - loss: 50886.0000
Epoch 2552/10000
1/1 [==============================] - 0s 4ms/step - loss: 35562.9414
Epoch 2553/10000
1/1 [==============================] - 0s 5ms/step - loss: 30479.4062
Epoch 2554/10000
1/1 [==============================] - 0s 4ms/step - loss: 40770.6094
Epoch 2555/10000
1/1 [==============================] - 0s 4ms/step - loss: 40687.2852
Epoch 2556/10000
1/1 [==============================] - 0s 5ms/step - loss: 35863.2109
Epoch 2557/10000
1/1 [==============================] - 0s 4ms/step - loss: 37504.6914
Epoch 2558/10000
1/1 [==============================] - 0s 4ms/step - loss: 39666.9531
Epoch 2559/10000
1/1 [==============================] - 0s 4ms/step - loss: 51908.4375
Epoch 2560/10000
1/1 [==============================] - 0s 4ms/step - loss: 27947.3027
Epoch 2561/10000
1/1 [==============================] - 0s 4

In [23]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 602ms/step
[1529.052](test_target) - [1369.1431](prediction) = 159.9089335937499


In [24]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.10458044173366891

In [25]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [26]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pará - IDH,Pará - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
0,0.930692,-0.953382,-1.132571,-0.920831,-0.879043,-0.573136,-1.217812,1.148205,-1.122043,-1.121418,...,-1.240316,-1.230476,-0.924464,1.397253,1.325002,1.044677,0.848136,-1.399021,1.223487,-1.294802
1,0.456803,-0.427906,-0.167173,-0.469129,-0.519886,1.406227,-0.013750,0.140893,-0.184474,-0.185492,...,0.031761,0.011544,-0.464604,-0.509514,-0.234391,0.303183,0.555982,0.878548,0.002512,0.154840
2,-1.387495,1.381289,1.299744,1.389960,1.398929,-0.833091,1.231562,-1.289098,1.306518,1.306910,...,1.208555,1.218932,1.389068,-0.887739,-1.090610,-1.347861,-1.404118,0.520474,-1.225999,1.139962


train_target:


,Pará - Consumo de Cimento (t)
0,1355.544
1,1640.682
2,1856.176


test_input:


...

,Pará - IDH Renda,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Desemprego,Pará - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pará - IDH,Pará - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pará - IDH Longevidade
12,-0.819242,0.814133,-0.996062,1.239044,0.97042,0.595754,1.119834,1.108405,2.635823,-0.189959,...,0.911811,-1.695891,1.553086,0.035714,-0.656689,-0.219776,2.805206,3.026913,-1.023926,-0.677741


test_target:


,Pará - Consumo de Cimento (t)
12,1529.052


1/1 [==============================] - 1s 550ms/step
Error: 159.9089335937499




[1905.6798095703125,
 1983.1087646484375,
 1332.21923828125,
 2001.510986328125,
 1794.6666259765625,
 1756.954345703125,
 1620.7998046875,
 1392.947998046875,
 1277.1947021484375,
 1369.14306640625]

In [27]:
display(targets)
display(predictions)

[2042.91,
 2318.608,
 2041.730714285714,
 1863.549771428571,
 1705.6805142857138,
 1564.94,
 1375.281,
 1290.928,
 1444.547,
 1529.052]

[1905.6798095703125,
 1983.1087646484375,
 1332.21923828125,
 2001.510986328125,
 1794.6666259765625,
 1756.954345703125,
 1620.7998046875,
 1392.947998046875,
 1277.1947021484375,
 1369.14306640625]

In [28]:
mae = mean_absolute_error(predictions, targets)
mae

227.60026082589292

In [29]:
porcentage = mae/np.mean(targets)
porcentage

0.1325011661229679